In [15]:
import torch

meanings_A = ["BAA", "ABA", "ABB", "BAB", "BBA"]
meanings_B = ["112", "221", "212", "122"]
Y = ["There is no (A,1) pair", "1st position", "2nd position", "3rd position"] # the question we try to answer is what is the position of the A,1 pair.
prior = torch.tensor([
    [[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0]],
    [[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0]],
    [[0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0]],
    ])
initial_prior = prior * 1/len(meanings_A) * 1/len(meanings_B)

utterances_A = ["1st position", "2nd position", "3rd position"]
utterances_B = ["1st position", "2nd position", "3rd position"]

lexicon_A = torch.tensor([
    [0, 1, 1], 
    [1, 0, 1], 
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    ])

lexicon_B = torch.tensor([
    [1, 1, 0], 
    [0, 0, 1], 
    [0, 1, 0],
    [1, 0, 0],
    ])

initial_lexica = [lexicon_A, lexicon_B]

print(f'prior: {prior}\n')
print(f'lexicon_A: {lexicon_A}\n')
print(f'lexicon_B: {lexicon_B}\n')

prior: tensor([[[0.0000, 0.0000, 0.0500, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0500, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0500, 0.0000, 0.0000]],

        [[0.0000, 0.0500, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0500, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]],

        [[0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]]])

lexicon_A: tensor([[0, 1, 1],
        [1, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

lexicon_B: ten

In [8]:
print(torch.exp(torch.log(initial_prior)))

tensor([[[0.0000, 0.0000, 0.0500, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0500, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0000, 0.0500, 0.0000, 0.0000]],

        [[0.0000, 0.0500, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0500, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0500, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]],

        [[0.0500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0500],
         [0.0500, 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0000, 0.0000, 0.0000]]], device='cuda:0')


In [10]:
from main import main
import torch
import random

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Model of the game
meanings_A = ["BAA", "ABA", "ABB", "BAB", "BBA"]
meanings_B = ["112", "221", "212", "122"]
utterances_A = ["1st position", "2nd position", "3rd position"]
utterances_B = ["1st position", "2nd position", "3rd position"]
Y = ["There is no (A,1) pair", "1st position", "2nd position", "3rd position"] # the question we try to answer is what is the position of the A,1 pair.
game_model = {"mA": meanings_A, "mB": meanings_B, "u": utterances_A, "v": utterances_B, "y": Y}

# Define the prior
prior = torch.tensor([
    [[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0]],
    [[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0]],
    [[0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]],
    [[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]],
    [[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0]],
    ], device=device, dtype=torch.float32)

initial_prior = prior * 1/len(meanings_A) * 1/len(meanings_B)

# Define the lexica
lexicon_A = torch.tensor([
    [0, 1, 1], 
    [1, 0, 1], 
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    ], device=device, dtype=torch.float32)

lexicon_B = torch.tensor([
    [1, 1, 0], 
    [0, 0, 1], 
    [0, 1, 0],
    [1, 0, 0],
    ], device=device, dtype=torch.float32)

initial_lexica = [lexicon_A, lexicon_B]

# Fix the meanings observed by the agents
# Define the parameters
meanings = [random.randint(0, len(meanings_A)-1), random.randint(0, len(meanings_B)-1)]
# meanings = [3, 1]

alpha = 1
number_of_rounds = 10
RSA_depth = 1
verbose = False 

main(initial_lexica, initial_prior, meanings, game_model, alpha, number_of_rounds, RSA_depth, device, verbose)

2024-09-05 19:15:49,026 - INFO - Agent A observes: BBA / id=4 and Agent B observes: 212 / id=2
2024-09-05 19:15:49,087 - INFO - Estimated meaning by Agent B: There is no (A,1) pair
2024-09-05 19:15:49,092 - INFO - Estimated meaning by Agent A: There is no (A,1) pair


Using device: cuda


In [12]:
import random

X = torch.tensor([[[ 1.0000e+00,  1.0000e+00, -0.0000e+00],
         [ 8.2038e-07,  8.2219e-07,  4.0365e-01],
         [ 1.2156e-06,  1.2116e-06,  5.9635e-01],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]],
        [[-1.2508e-15, -1.2508e-15, -1.2508e-15],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-5.5305e-06, -5.5305e-06, -1.1249e-11]],
        [[-2.1081e-16, -2.1081e-16, -2.3841e-26],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-2.5626e-22, -2.5543e-22, -2.5575e-22],
         [-1.7292e-22, -1.7330e-22, -1.7308e-22]],
        [[ 1.0426e+01,  1.0426e+01, -1.3419e-06],
         [ 8.5534e-06,  8.5723e-06, -9.0826e-07],
         [-0.0000e+00, -0.0000e+00,  0.0000e+00],
         [-0.0000e+00, -0.0000e+00,  0.0000e+00]]], device='cuda:0')
argmaxes = torch.where(X[0,:,2] == torch.max(X[0,:,2]))[0]
argmaxes

tensor([2], device='cuda:0')

In [21]:
prior = torch.tensor([[[0.2, 0.5, 0.5, 0.1],
                       [0.1, 0.5, 0.5, 0.0]],
                       [[0.2, 0.1, 0.1, 0.25],
                       [0.2, 0, 0., 0.25]],
                       [[0.25, 0.25, 0.25, 0.25],
                       [0.25, 0.25, 0.25, 0.25]]])
print(prior.sum(dim=2, keepdim=True).sum(dim=0, keepdim=True) + 1e-10)
prior/(prior.sum(dim=2, keepdim=True).sum(dim=0, keepdim=True) + 1e-10)

tensor([[[2.9500],
         [2.5500]]])


tensor([[[0.0678, 0.1695, 0.1695, 0.0339],
         [0.0392, 0.1961, 0.1961, 0.0000]],

        [[0.0678, 0.0339, 0.0339, 0.0847],
         [0.0784, 0.0000, 0.0000, 0.0980]],

        [[0.0847, 0.0847, 0.0847, 0.0847],
         [0.0980, 0.0980, 0.0980, 0.0980]]])